In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, ActivityRegularization
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.regularizers import l1, l2
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD, Adadelta


Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print("GPU device name is: " + tf.test.gpu_device_name())
print(tf.__version__)
K.tensorflow_backend._get_available_gpus()


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3007197650656760835
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5159190528
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2746153569715854015
physical_device_desc: "device: 0, name: GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
GPU device name is: /device:GPU:0
1.10.0


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
def recur(i,j,train_images1,detected):        
        detected.append([i,j])
        if i + 1 < 63 and [i+1,j] not in detected:
            if train_images1[i+1][j] == 1:
                recur(i+1,j,train_images1,detected)
        if j + 1 < 63 and [i,j+1] not in detected:
            if train_images1[i][j+1] == 1:
                recur(i,j+1,train_images1,detected)
        if i - 1 > 0 and [i-1,j] not in detected:
            if train_images1[i - 1][j] == 1:
                recur(i-1,j,train_images1,detected)        
        if j - 1 > 0 and [i,j-1] not in detected:
            if train_images1[i][j - 1] == 1:
                recur(i,j - 1,train_images1,detected)
    

def preprocess(input_images):       

    croped = []
    croped_image = []
    for img_index in range(len(input_images)):
        print("processing image " + str(img_index)+ " of " + str(len(input_images)), end= '\r')
        input_images1= input_images[img_index] > 250
    
        
        obj = [] 
        obj_number = -1              
        for i in range(64):
            for j in range(64):
                detected = []
                if obj_number == -1:
                    if input_images1[i][j] == 1:
                        recur(i,j,input_images1,detected)
                        down = max([_[0] for _ in detected])
                        up = min([_[0] for _ in detected])
                        right = max([_[1] for _ in detected])
                        left = min([_[1] for _ in detected])

                        if down-up+1 > right-left+1:
                            area =  (down-up+1) **2
                        elif down-up+1 <= right-left+1:
                             area =  (right-left+1) **2
                        obj.append([up,down,left,right,area])
                        obj_number += 1
                
                elif input_images1[i][j] == 1 and not (i in range(obj[obj_number][0],obj[obj_number][1]+1) and j in range(obj[obj_number][2],obj[obj_number][3]+1)) :
                    recur(i,j,input_images1,detected)
                    down = max([_[0] for _ in detected])
                    up = min([_[0] for _ in detected])
                    right = max([_[1] for _ in detected])
                    left = min([_[1] for _ in detected])
                  
                    if down-up+1 > right-left+1:
                            area =  (down-up+1) **2
                    elif down-up+1 <= right-left+1:
                            area =  (right-left+1) **2   
                    obj.append([up,down,left,right,area])
                    obj_number += 1
                
                
        maximum = -1 
        for i in range(len(obj)):
                    
            if obj[i][4] > maximum:
                maximum = obj[i][4]
                index = i        
    
        croped.append(obj[index])
        
        croped_image.append(input_images1[obj[index][0]:obj[index][1]+1,obj[index][2]:obj[index][3]+1])
        
        while np.shape(croped_image[img_index])[0] < 64:
           croped_image[img_index] = np.insert(croped_image[img_index],0,np.zeros(np.shape(croped_image[img_index])[1]),axis = 0)
           if np.shape(croped_image[img_index])[0] < 64:
                croped_image[img_index] = np.insert(croped_image[img_index],np.shape(croped_image[img_index])[0],np.zeros(np.shape(croped_image[img_index])[1]),axis = 0)
        
        while np.shape(croped_image[img_index])[1] < 64:
            croped_image[img_index] = np.insert(croped_image[img_index],0,np.zeros(np.shape(croped_image[img_index])[0]),axis = 1)
            if np.shape(croped_image[img_index])[1] < 64:
               croped_image[img_index] = np.insert(croped_image[img_index],np.shape(croped_image[img_index])[1],np.zeros(np.shape(croped_image[img_index])[0]),axis = 1)
    
    croped_image = np.asarray(croped_image)
    
    return croped_image

In [3]:
# Data preprocessing and splitting 

num_classes = 10
x_train = pd.read_pickle('./datasets/train_processed1.pkl')
# x_train = np.load('./datasets/train_processed1.npy')
# x_test = pd.read_pickle('./datasets/test_images.pkl')
Y_train = pd.read_csv('./datasets/train_labels.csv')
x_test = np.load('./datasets/test_processed1.npy')

# Preprocessing 

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255  
x_train = np.asarray(x_train)
# x_train = preprocess(x_train)


# x_test = pd.read_pickle('./datasets/test_images.pkl')
# x_test = preprocess(x_test)

# chage csv format to numpy array for training labels

y_train = []
for i in range(x_train.shape[0]):
    y_train.append(Y_train.iloc[i]['Category'])
    
y_train = np.asarray(y_train)
y_train = y_train.reshape(np.shape(y_train)[0],1)
y_train = keras.utils.to_categorical(y_train, num_classes)

# Reshaping the array to 4-dims so that it can work with the Keras API

img_rows = x_train.shape[1]
img_cols = x_train.shape[2]

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
X_Test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# Splitting training data into Train-Dev 

X_Training, X_Validation, Y_Training, Y_Validation = train_test_split(x_train, y_train, test_size = 0.0625)

print('X_Training shape:', X_Training.shape)
print('X_Validation shape:', X_Validation.shape)
print('Y_Training shape:', Y_Training.shape)
print('Y_Validation shape:', Y_Validation.shape)
print('X_Test shape:', X_Test.shape)
print('Number of images in training set', X_Training.shape[0])
print('Number of images in validation set', X_Validation.shape[0])
print('Number of images in test set', X_Test.shape[0])


X_Training shape: (37500, 64, 64, 1)
X_Validation shape: (2500, 64, 64, 1)
Y_Training shape: (37500, 10)
Y_Validation shape: (2500, 10)
X_Test shape: (10000, 64, 64, 1)
Number of images in training set 37500
Number of images in validation set 2500
Number of images in test set 10000


In [4]:
# Original mnist data
from keras.datasets import mnist

(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()
img_rows_mnist, img_cols_mnist = x_train_mnist.shape[1], x_train_mnist.shape[1]

    
x_train_mnist = x_train_mnist.astype('float32')
x_test_mnist = x_test_mnist.astype('float32')
x_train_mnist /= 255
x_test_mnist /= 255    

y_train_mnist = keras.utils.to_categorical(y_train_mnist, num_classes)
y_test_mnist = keras.utils.to_categorical(y_train_mnist, num_classes)

train = np.zeros((x_train_mnist.shape[0],img_rows,img_cols))
test = np.zeros((x_test_mnist.shape[0],img_rows,img_cols))

for i in range(x_train_mnist.shape[0]):
    train[i,:,:] = np.pad(x_train_mnist[i,:,:],pad_width= 18, mode= 'constant', constant_values=0)
    
for i in range(x_test_mnist.shape[0]):
    test[i,:,:] = np.pad(x_test_mnist[i,:,:],pad_width= 18, mode= 'constant', constant_values=0)

x_train_mnist = train.reshape(x_train_mnist.shape[0], img_rows, img_cols,1)
x_test_mnist = test.reshape(x_test_mnist.shape[0], img_rows, img_cols,1)
    
print('x_train_mnist shape:', x_train_mnist.shape)
print(x_train_mnist.shape[0], 'mnist train samples')
print(x_test_mnist.shape[0], 'mnist test samples')

x_train_mnist shape: (60000, 64, 64, 1)
60000 mnist train samples
10000 mnist test samples


In [6]:

# Importing the required Keras modules containing model and layers
def model1():

    # Creating a Sequential Model and adding the layers
    model = Sequential()
    
    model.add(Conv2D(128, kernel_size=(5,5)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.10))
    
    model.add(Conv2D(64, kernel_size=(5,5)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
    
    model.add(Conv2D(32, kernel_size=(3,3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
    

    
    model.add(Conv2D(32, kernel_size=(4,4)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
    
#     model.add(Conv2D(1024, kernel_size=(3,3)))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Dropout(0.25))
    model.add(Flatten()) # Flattening the 2D arrays for fully connected layers

    model.add(Dense(1024, activation=tf.nn.relu))
#     model.add(Dropout(0.3))

#     model.add(Dense(128, activation=tf.nn.relu))

    model.add(Dropout(0.5))

    model.add(Dense(10,activation=tf.nn.softmax)) # Since we have ten classes
    
    return model

In [9]:

# Importing the required Keras modules containing model and layers
def model2():

    # Creating a Sequential Model and adding the layers
    model = Sequential()
    
    model.add(Conv2D(128, kernel_size=(7,7)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, kernel_size=(5,5)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.05))
    
    model.add(Conv2D(32, kernel_size=(3,3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
    

    
#     model.add(Conv2D(128, kernel_size=(4,4)))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
    
#     model.add(Conv2D(1024, kernel_size=(3,3)))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Dropout(0.25))
    model.add(Flatten()) # Flattening the 2D arrays for fully connected layers

    model.add(Dense(1024, activation=tf.nn.relu))
#     model.add(Dropout(0.3))

#     model.add(Dense(128, activation=tf.nn.relu))

    model.add(Dropout(0.5))

    model.add(Dense(10,activation=tf.nn.softmax)) # Since we have ten classes
    
    return model

In [12]:

# Importing the required Keras modules containing model and layers
def model3():

    # Creating a Sequential Model and adding the layers
    model = Sequential()
    
    model.add(Conv2D(128, kernel_size=(3,3),input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, kernel_size=(3,3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, kernel_size=(3,3)))
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    model.add(Dropout(0.25))

#     model.add(Conv2D(128, kernel_size=(3,3)))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
   
#     model.add(Conv2D(128, kernel_size=(3,3)))
#     model.add(MaxPooling2D(pool_size=(2, 2))) 
#     model.add(ActivityRegularization(l1=0.01, l2=0.01))
    model.add(Flatten()) # Flattening the 2D arrays for fully connected layers

#     model.add(Dense(1024, activation=tf.nn.relu))
#     model.add(Dropout(0.3))
    model.add(Dense(1024, activation=tf.nn.relu))
    
    model.add(Dropout(0.5))
    
    model.add(Dense(10,activation=tf.nn.softmax)) # Since we have ten classes
    
    return model

In [15]:
augment_size = 30000
image_generator = ImageDataGenerator(
            rotation_range=10,
            zoom_range = 0.05, 
            width_shift_range=0.05,
            height_shift_range=0.05,
            horizontal_flip=False,
            vertical_flip=False)
# fit data for zca whitening
image_generator.fit(X_Training, augment=True)
# get transformed images
randidx = np.random.randint(X_Training.shape[0], size=augment_size)
x_augmented = X_Training[randidx].copy()
y_augmented = Y_Training[randidx].copy()
x_augmented = image_generator.flow(x_augmented, np.zeros(augment_size),
                            batch_size=augment_size, shuffle=False).next()[0]
# append augmented data to trainset
X_Tr = np.concatenate((X_Training, x_augmented))
Y_Tr = np.concatenate((Y_Training, y_augmented))
print(X_Tr.shape[0])


67500


In [24]:
model = model3()
epochs = 20
batch_size = 128

model.compile(optimizer='Adadelta', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


model.fit(x=X_Training, y=Y_Training, epochs = epochs, batch_size= batch_size)

# Evaluate the model on validation set
Y_Prediction = model.predict_classes(x= X_Validation, batch_size= batch_size)
Y_Prediction = keras.utils.to_categorical(Y_Prediction, num_classes)
print('')
print('Validation Accuracy for CNN is: ') 
accuracy_score(Y_Validation, Y_Prediction)

Epoch 1/20
37500/37500 [==============================] - 13s 347us/step - loss: 0.6760 - acc: 0.7931
Epoch 2/20
37500/37500 [==============================] - 12s 325us/step - loss: 0.3727 - acc: 0.8985
Epoch 3/20
37500/37500 [==============================] - 12s 328us/step - loss: 0.3193 - acc: 0.9155
Epoch 4/20
37500/37500 [==============================] - 12s 330us/step - loss: 0.2885 - acc: 0.9226
Epoch 5/20
37500/37500 [==============================] - 13s 336us/step - loss: 0.2647 - acc: 0.9289
Epoch 6/20
37500/37500 [==============================] - 13s 334us/step - loss: 0.2432 - acc: 0.9348
Epoch 7/20
37500/37500 [==============================] - 13s 345us/step - loss: 0.2294 - acc: 0.9398
Epoch 8/20
37500/37500 [==============================] - 12s 332us/step - loss: 0.2146 - acc: 0.9411
Epoch 9/20
37500/37500 [==============================] - 12s 332us/step - loss: 0.2008 - acc: 0.9453
Epoch 10/20
37500/37500 [==============================] - 12s 325us/step - loss: 

0.952

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 62, 62, 128)       1280      
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 29, 29, 64)        73792     
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 6, 6, 64)          0         
__________

In [ ]:
model = model4()
epochs = 20
batch_size = 128

model.compile(optimizer='adadelta', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
iterations= 3
validation = np.zeros((1,Y_Training.shape[1]))

# for i in range(0,iterations):

model.fit(x=X_Training, y=Y_Training, epochs = epochs, batch_size= batch_size,shuffle=True)

middle = model.predict(x= X_Validation, batch_size= batch_size)
# validation = validation + middle

model1 = 
    
# Y_Prediction = keras.utils.to_categorical(validation, num_classes)


In [ ]:
# Evaluate the model on validation set
Y_Prediction = model.predict_classes(x= X_Validation, batch_size= batch_size)
Y_Prediction = keras.utils.to_categorical(Y_Prediction, num_classes)
print('')
print('Validation Accuracy for CNN is: ') 
accuracy_score(Y_Validation, Y_Prediction)

In [ ]:
# Transfer Learning from original mnist model
mymodel = model
for layer in mymodel.layers[:-5]:
    layer.trainable = False

for layer in mymodel.layers:
    print(layer, layer.trainable)# Adding custom layers
    
final_model = Sequential()
final_model.add(mymodel)
# final_model.add(Flatten())
# final_model.add(Dense(128, activation='relu'))
# final_model.add(Dropout(0.5))
# final_model.add(Dense(num_classes, activation='softmax'))
final_model.summary()

# final_model.compile(optimizer='adam', 
#               loss='categorical_crossentropy', 
#               metrics=['accuracy'])
# final_model.fit(x=x_train, y=y_train, epochs = epochs, batch_size= batch_size)

In [ ]:
print(input_shape)
epochs = 15
batch_size = 128

final_model.compile(optimizer='adadelta', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
final_model.fit(x=X_Training, y=Y_Training, epochs = epochs, batch_size= batch_size)

In [ ]:
print(input_shape)
model_Deep = model2()
epochs_Deep = 5
batch_size = 128

model_Deep.compile(optimizer='adadelta', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model_Deep.fit(x=x_train, y=y_train, epochs = epochs_Deep, batch_size= batch_size)

In [17]:
# Function to write test predictions to .csv file

def csvWriter(prediction, submission_no):
    index = 0
    filename = 'G_40_submission_' + str(submission_no) + '.csv'
    csv = open(filename, "w") 

    columnTitleRow = "Id,Category\n"
    csv.write(columnTitleRow)
    
    for i in prediction:
        csv.write(str(index) + ',' + str(i) + "\n")
        index+=1
    
    csv.close()
    
# Test set prediction and creating csv for Kaggle submission

Y_Test = model.predict_classes(X_Test, batch_size= batch_size)
csvWriter(Y_Test,9)

In [ ]:
# inividual classification


image_index = 830
plt.imshow(x_test[image_index].reshape(img_rows, img_cols),cmap='Greys')
pred = model.predict(x_test[image_index].reshape(1, img_rows, img_cols, 1))
print(pred.argmax())

In [ ]:
md = X_Training.reshape(X_Training.shape[0],X_Training.shape[1],X_Training.shape[2])
td = X_Validation.reshape(X_Validation.shape[0],X_Validation.shape[1],X_Validation.shape[2])
xtr = np.zeros((X_Training.shape[0],X_Training.shape[1],X_Training.shape[2],3))
xv = np.zeros((X_Validation.shape[0],X_Validation.shape[1],X_Validation.shape[2],3))

for i in range(X_Training.shape[0]):
    for j in range(3):
        xtr[i,:,:,j] = md[i,...]
        
for i in range(X_Validation.shape[0]):
    for j in range(3):
        xv[i,:,:,j] = td[i,...]       
        
        
print(xtr.shape)
print(xv.shape)

In [ ]:
from keras.applications import VGG16

input_sh = X_Training[0,...]

vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=input_sh)

model1 = models.Sequential()
model1.add(vgg_conv)
model1.add(layers.Dense(256, activation='relu'))
model1.add(layers.Dropout(0.5))
model1.add(layers.Dense(3, activation='softmax'))


model1.compile(optimizer=optimizers.Adam(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])
 
history = model.fit(xtr,
                    Y_Training,
                    epochs=20,
                    batch_size=batch_size,
                    validation_data=(xv,Y_Validation))

Y_Prediction = model.predict_classes(x= xv, batch_size= batch_size)
Y_Prediction = keras.utils.to_categorical(Y_Prediction, num_classes)
print('')
print('Validation Accuracy for CNN is: ') 
accuracy_score(Y_Validation, Y_Prediction)

In [ ]:
X_Training[30,...].shape